Data Preparation for fine-tuning

used two datasets - 'heliosbrahma/mental_health_chatbot_dataset' and 'mpingale/mental-health-chat-dataset'.

converted both the datasets to a single column "text", containing prompt-response pairs.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import re
from datasets import Dataset

In [ ]:
from datasets import load_dataset
dataset1 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

(…)-00000-of-00001-01391a60ef5c00d9.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

In [ ]:
dataset1['train']['text']

['<HUMAN>: What is a panic attack?\n<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.',
 "<HUMAN>: What are symptoms of panic attack vs. anxiety attack?\n<ASSISTANT>: Panic attacks and anxiety attacks can share some similarities, but they also have distinct features. It's important to remember that I am not a medical professional, and if you or someone you know is experiencing these symptoms, seeking help from a qualified healthcare professional is essential.\n\nPanic Attack Symptoms:\nA panic attack is a sudden and intense surge of fear

In [ ]:
def reformat_text(row):
    match = re.match(r"<HUMAN>: (.*?)\n<ASSISTANT>: (.*)", row['text'])
    if match:
        prompt, response = match.groups()
        return {"text": f"<PROMPT>: {prompt} <RESPONSE>: {response}"}
    return row

dataset1 = dataset1.map(reformat_text)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
dataset1["train"][:10]

{'text': ['<PROMPT>: What is a panic attack? <RESPONSE>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.',
  "<PROMPT>: What are symptoms of panic attack vs. anxiety attack? <RESPONSE>: Panic attacks and anxiety attacks can share some similarities, but they also have distinct features. It's important to remember that I am not a medical professional, and if you or someone you know is experiencing these symptoms, seeking help from a qualified healthcare professional is essential.",
  '<PROMPT>: What are the types of Mental Illness? <RESPONSE>: There 

In [ ]:
df1 = dataset1["train"].to_pandas()

In [ ]:
df1.head()

,text
0,<PROMPT>: What is a panic attack? <RESPONSE>: ...
1,<PROMPT>: What are symptoms of panic attack vs...
2,<PROMPT>: What are the types of Mental Illness...
3,<PROMPT>: What does mental-illness mean? <RESP...
4,<PROMPT>: How can you treat mental illness? <R...


In [ ]:
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")

In [ ]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views', 'text'],
        num_rows: 2775
    })
})

In [ ]:
dataset2['train']['text']

['<s>[INST] I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling? [/INST] It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be helped" or that they will "be too much for their therapist." I don\'t know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,\xa0 people have numerous significant stressors in their lives.\xa0 Let\'s face it, life can be complicated! Therapists are completely ready and equipped to handle 

In [ ]:
import re

def reformat_texts(row):
    cleaned_text = row['text'].replace("\xa0", " ")
    match = re.match(r"<s>\[INST\](.*?)\[/INST\](.*?)</s>", cleaned_text, re.DOTALL)
    if match:
        content = match.group(1).strip()
        prompt, response = match.groups()
        return {"text": f"<PROMPT>: {prompt.strip()} <RESPONSE>: {response.strip()}"}
    return row

dataset2 = dataset2.map(reformat_texts)


Map:   0%|          | 0/2775 [00:00<?, ? examples/s]

In [ ]:
dataset2['train']['text']

['<PROMPT>: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling? <RESPONSE>: It is very common for people to have multiple issues that they want to (and need to) address in counseling.  I have had clients ask that same question and through more exploration, there is often an underlying fear that they  "can\'t be helped" or that they will "be too much for their therapist." I don\'t know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,  people have numerous significant stressors in their lives.  Let\'s face it, life can be complicated! Therapists are completely ready and equipped to handle all of the 

In [ ]:
dataset2["train"][:10]

{'questionID': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'questionTitle': ['Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?',
  'Do I have too many issues for counseling?'],
 'questionText': ['I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?',
  'I have so many issues to address. I have a history of sexual abu

In [ ]:
df2 = dataset2["train"].to_pandas()

In [ ]:
df2.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,text
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971,<PROMPT>: I have so many issues to address. I ...
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386,<PROMPT>: I have so many issues to address. I ...
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071,<PROMPT>: I have so many issues to address. I ...
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643,<PROMPT>: I have so many issues to address. I ...
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256,<PROMPT>: I have so many issues to address. I ...


In [ ]:
df2 = df2['text']

In [ ]:
df2.head()

,text
0,<PROMPT>: I have so many issues to address. I ...
1,<PROMPT>: I have so many issues to address. I ...
2,<PROMPT>: I have so many issues to address. I ...
3,<PROMPT>: I have so many issues to address. I ...
4,<PROMPT>: I have so many issues to address. I ...


In [ ]:
df2=df2.to_frame(name="text")

In [ ]:
print(type(df2))


<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(type(df1))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(df2.columns)

Index(['text'], dtype='object')


In [ ]:
df2 = df2[~df2['text'].str.contains("<PROMPT>: None")]

In [ ]:
df2

,text
0,<PROMPT>: I have so many issues to address. I ...
1,<PROMPT>: I have so many issues to address. I ...
2,<PROMPT>: I have so many issues to address. I ...
3,<PROMPT>: I have so many issues to address. I ...
4,<PROMPT>: I have so many issues to address. I ...
...,...
2770,<PROMPT>: What are some difficulties that a co...
2771,<PROMPT>: What are some difficulties that a co...
2772,<PROMPT>: What are some difficulties that a co...
2773,<PROMPT>: What are some difficulties that a co...


In [ ]:
final_data = pd.concat([df1, df2])

In [ ]:
final_data

,text
0,<PROMPT>: What is a panic attack? <RESPONSE>: ...
1,<PROMPT>: What are symptoms of panic attack vs...
2,<PROMPT>: What are the types of Mental Illness...
3,<PROMPT>: What does mental-illness mean? <RESP...
4,<PROMPT>: How can you treat mental illness? <R...
...,...
2770,<PROMPT>: What are some difficulties that a co...
2771,<PROMPT>: What are some difficulties that a co...
2772,<PROMPT>: What are some difficulties that a co...
2773,<PROMPT>: What are some difficulties that a co...


In [ ]:
fine_tune_data = final_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
fine_tune_data

,text
0,<PROMPT>: My boyfriend and I have been togethe...
1,"<PROMPT>: I'm transgender, I know I am, but I'..."
2,<PROMPT>: I suffer from checking and cleaning ...
3,<PROMPT>: My girlfriend was abused as a child....
4,<PROMPT>: My daughter is in later elementary s...
...,...
2803,<PROMPT>: I love my girlfriend so much. I get ...
2804,<PROMPT>: When they come home from their dad's...
2805,<PROMPT>: The birth mother attempted suicide s...
2806,<PROMPT>: I know that I need to get past my fe...


In [ ]:
fine_tune_data.to_excel('data_5983_final.xlsx', index=False)